In [ ]:
import glob
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from endstate_rew.constant import kBT
from endstate_rew.neq import perform_switching, perform_inst_switching
from endstate_rew.system import generate_molecule, initialize_simulation
from openmm import unit
from pymbar import BAR, EXP

warnings.simplefilter("ignore")
###############
###############
name = 'acetylacetone'
smiles = 'CC(C(C)=O)C(C)=O'
switching_length = 5_001
###############
w_dir = f'{name}_neq_switching'


In [ ]:
def _get_samples(direction:str = 'mm_to_qml'):
    files = glob.glob(f"../data/{w_dir}/{name}_*_{direction}_{name}_*_{switching_length}*.pickle")
    ws = []
    for f in files:
        w_ = pickle.load(open(f, 'rb')).value_in_unit(unit.kilojoule_per_mole)
        ws.extend(w_)
    number_of_samples = len(ws)
    print(f"Number of samples used: {number_of_samples}")
    return ws * unit.kilojoule_per_mole

In [ ]:
ws_from_mm_to_qml = np.array(_get_samples('mm_to_qml')/ kBT)
ws_from_qml_to_mm = np.array(_get_samples('qml_to_mm')/ kBT)

In [ ]:
print(f"Crooks' equation: {BAR(ws_from_mm_to_qml, ws_from_qml_to_mm)}")
print(f"Jarzynski's equation: {EXP(ws_from_mm_to_qml)}")

In [ ]:
# instantenious swichting (FEP)
mm_samples = pickle.load(open(f'../data/{name}_mm_samples_2000_1000.pickle', 'rb'))
qml_samples = pickle.load(open(f'../data/{name}_qml_samples_2000_1000.pickle', 'rb'))

switching_length = 2
nr_of_switches = 500
# create molecule
molecule = generate_molecule(smiles)
sim = initialize_simulation(molecule)
lambs = np.linspace(0,1,switching_length)
dEs_from_mm_to_qml = perform_inst_switching(sim, lambs, samples=mm_samples,nr_of_switches=nr_of_switches)
lambs = np.linspace(1,0,switching_length)
dEs_from_qml_to_mm = perform_inst_switching(sim,lambs, samples=qml_samples,nr_of_switches=nr_of_switches)
print()
print(f"Number of samples used: {len(dEs_from_mm_to_qml)}")
print(f"Number of samples used: {len(dEs_from_mm_to_qml)}")

# to kT
dEs_from_mm_to_qml = np.array(dEs_from_mm_to_qml/ kBT)
dEs_from_qml_to_mm = np.array(dEs_from_qml_to_mm/ kBT)
print(f'FEP: From MM to QML: {EXP(dEs_from_mm_to_qml)}')
print(f'FEP: From QML to MM: {EXP(dEs_from_qml_to_mm)}')
print(f"BAR: {BAR(dEs_from_mm_to_qml, dEs_from_qml_to_mm)}")

In [ ]:
sns.set_context("talk")
fig, axs = plt.subplots(1,2, figsize=(10.0, 5.5), dpi=600)
palett = sns.color_palette(n_colors=8)
palett_as_hex = palett.as_hex()
c1, c2 = palett_as_hex[0], palett_as_hex[1]

sns.histplot(ax=axs[0], data=ws_from_mm_to_qml, kde=True, stat='density', label=r'dW', color=c1)
sns.histplot(ax=axs[0], data=dEs_from_mm_to_qml, kde=True, stat='density', label=r'dE', color=c2)
sns.histplot(ax=axs[1], data=ws_from_qml_to_mm, kde=True, stat='density', label=r'dW', color=c1)
sns.histplot(ax=axs[1], data=dEs_from_qml_to_mm, kde=True, stat='density', label=r'dE', color=c2)
axs[0].set_title(r'MM$\rightarrow$QML')
axs[1].set_title(r'QML$\rightarrow$MM')
axs[0].legend()
axs[1].legend()

plt.show()

In [ ]:
sns.histplot()